In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
# Cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
lefteye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')
righteye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_righteye_2splits.xml')

In [ ]:
# load model
model = tf.keras.models.load_model('train_dd.h5')

In [ ]:
cap = cv2.VideoCapture(0)

FONT = cv2.FONT_HERSHEY_SIMPLEX

while True:
    _, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    left_eyes = lefteye_cascade.detectMultiScale(img)
    right_eyes = righteye_cascade.detectMultiScale(img)
    all_eyes = [*left_eyes, *right_eyes]
    status= ''
    
    if left_eyes == () and right_eyes == ():
        text = "Eyes not detected!"
    else:
        # print(len(all_eyes))
        preds, img_size = [], 224
        
        for i, (x,y,w,h) in enumerate(all_eyes):
            # print(i, (x, y, w, h))
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            # plt.imshow(cv2.cvtColor(roi_color, cv2.COLOR_BGR2RGB))
            new_array = cv2.resize(roi_color, (img_size, img_size))
            x_input = np.array(new_array).reshape(1, img_size, img_size, 3)
            x_input = x_input / 255.0

            prediction = model.predict(x_input)
            preds += prediction,
            
            print(prediction)
            
        if any(p < 0.9 for p in preds):
            text = 'DROWSY'
            status = "Closed Eyes"
            cv2.putText(img, status, (50, 450), FONT, 3, (0, 0, 255), 2, cv2.LINE_4)
            cv2.rectangle(img, (x, x), (x + w, y + h), (0,0,0), -1)
            
        else:
            text = 'ACTIVE'
            status = "Open Eyes"
            cv2.putText(img, status, (50, 450), FONT, 3, (0, 255, 0), 2, cv2.LINE_4)
            x1, y1, w1, h1 = 0, 0, 175, 75
            
            # Notice on top-left
            cv2.rectangle(img, (x1, x1), (x1 + w1, y1 + h1), (0,0,0), -1)
            cv2.putText(img, text , (x1 + int(w1/10), y1 + int(h1 / 2)), 
                       FONT, 0.7, (0, 255, 0), 2)
            
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for x, y, w, h in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0, 0, 255), 2)
        
    # cv2.putText(img, status,(50, 50), FONT, 3, (0,0,255), 2, cv2.LINE_4)
    
    cv2.imshow('Drowsiness Detection', img)

    
    if cv2.waitKey(2) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
        

In [ ]:
cap.release()